# INSTALLING PACKAGES AND IMPORTING LIBRARIES

In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install -U sentence-transformers
!{sys.executable} -m pip install tqdm

Requirement already up-to-date: sentence-transformers in c:\users\augus\appdata\local\programs\python\python37\lib\site-packages (2.2.2)


You should consider upgrading via the 'c:\Users\augus\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Users\augus\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


In [2]:
import relief
import utils
import pandas as pd
import numpy as np
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

# Visualization
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
# Warnings
import warnings
warnings.filterwarnings('ignore')

c:\Users\augus\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# WOMEN E COMMERCE

In [3]:
df = pd.read_csv(r"C:\Users\augus\Desktop\projectTesis\datasets\women_e_commerce\Womens Clothing E-Commerce Reviews.csv")
# contain redundant information about the index of the dataset
df = df.drop(columns = "Unnamed: 0")
# reformat the column names
df.columns = (df.columns
                .str.replace(' ', '_')
                .str.lower()
             )

# Drop rows with missing values
df = df.dropna(subset=['review_text'])
#drop column review title.
df = df.drop(columns = "title")
#drop column review text.
df = df.drop(columns = "review_text")
#df['review_text'] = df['review_text'].astype(str)
# Replacing "Initmitates" with "Intimates" in 'text_column'
df['division_name'] = df['division_name'].str.replace('Initmates', 'Intimates')
df['department_name'] = df['department_name'].str.replace('Intimate', 'Intimates')

#X = df.sample(n=10000, ignore_index=True)

df.head()

,clothing_id,age,rating,recommended_ind,positive_feedback_count,division_name,department_name,class_name
0,767,33,4,1,0,Intimates,Intimates,Intimates
1,1080,34,5,1,4,General,Dresses,Dresses
2,1077,60,3,0,0,General,Dresses,Dresses
3,1049,50,5,1,0,General Petite,Bottoms,Pants
4,847,47,5,1,6,General,Tops,Blouses


In [4]:
df.dtypes

clothing_id                 int64
age                         int64
rating                      int64
recommended_ind             int64
positive_feedback_count     int64
division_name              object
department_name            object
class_name                 object
dtype: object

In [5]:
df['clothing_id'] = df['clothing_id'].astype("object")
df['recommended_ind'] = df['recommended_ind'].astype("category")
df.dtypes

clothing_id                  object
age                           int64
rating                        int64
recommended_ind            category
positive_feedback_count       int64
division_name                object
department_name              object
class_name                   object
dtype: object

In [6]:
# deep copy
data = df.copy()
# relief best attributes= rating. clothing_id (display significant Results)
relief_data = df[["clothing_id","rating", "recommended_ind"]]

# the label is recommmended_ind, the rest are the features.
y_relief = relief_data.pop("recommended_ind")
y_total = data.pop("recommended_ind")

In [7]:
relief_data.dtypes

clothing_id    object
rating          int64
dtype: object

In [8]:
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(relief_data)
categorical_columns = categorical_columns_selector(relief_data)

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

preprocessor = ColumnTransformer(
    [
        ("one-hot-encoder", categorical_preprocessor, categorical_columns),
        ("standard_scaler", numerical_preprocessor, numerical_columns),
    ]
)



model = make_pipeline(preprocessor, LogisticRegression(max_iter=500))
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['clothing_id']),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  ['rating'])])),
                ('logisticregression', LogisticRegression(max_iter=500))])

In [9]:
data_train, data_test, target_train, target_test = train_test_split(
    relief_data, y_relief, random_state=42)

In [10]:
_ = model.fit(data_train, target_train)

In [11]:
data_test.head()

,clothing_id,rating
13365,948,5
19834,875,4
18722,829,4
10635,1068,5
7348,1030,2


In [12]:
model.predict(data_test)[:5]

array([1, 1, 1, 1, 0], dtype=int64)

In [13]:
target_test[:5]

13365    1
19834    1
18722    1
10635    1
7348     1
Name: recommended_ind, dtype: category
Categories (2, int64): [0, 1]

In [14]:
# accuracy score on the entire test set.
model.score(data_test, target_test)

0.9309309309309309

For the total model.

In [15]:
numerical_columns_total = numerical_columns_selector(data)
categorical_columns_total = categorical_columns_selector(data)

preprocessor_total = ColumnTransformer(
    [
        ("one-hot-encoder", categorical_preprocessor, categorical_columns_total),
        ("standard_scaler", numerical_preprocessor, numerical_columns_total),
    ])
model_total = make_pipeline(preprocessor_total, LogisticRegression(max_iter=500))
model_total

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['clothing_id',
                                                   'division_name',
                                                   'department_name',
                                                   'class_name']),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  ['age', 'rating',
                                                   'positive_feedback_count'])])),
                ('logisticregression', LogisticRegression(max_iter=500))])

In [16]:
data_train, data_test, target_train, target_test = train_test_split(
    data, y_total, random_state=42)

In [17]:
_ = model_total.fit(data_train, target_train)
model_total.predict(data_test)[:5]

array([1, 1, 1, 1, 0], dtype=int64)

In [18]:
# accuracy score on the entire test set.
model_total.score(data_test, target_test)

0.9298710475181063

## cross validation of the results

In [19]:
cv_results = cross_validate(model, relief_data, y_relief, cv=5)
scores = cv_results["test_score"]
print(
    "The mean cross-validation accuracy is: "
    f"{scores.mean():.3f} ± {scores.std():.3f}"
)

The mean cross-validation accuracy is: 0.931 ± 0.002


In [20]:
cv_results = cross_validate(model_total, data, y_total, cv=5)
scores = cv_results["test_score"]
print(
    "The mean cross-validation accuracy is: "
    f"{scores.mean():.3f} ± {scores.std():.3f}"
)

The mean cross-validation accuracy is: 0.931 ± 0.002


# PET FINDER

In [21]:
# set the max columns to none
pd.set_option('display.max_columns', None)
df = pd.read_csv(r"C:\Users\augus\Desktop\projectTesis\datasets\train_pet_finder.csv")
df.head()

,Unnamed: 0,Unnamed: 0.1,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,10318,10318,2,Xiao Li,2,Domestic Short Hair,NaN,2,Brown,White,NaN,2,1,2,1,2,1,1,0,Selangor,744fa4278196568a18fcf5cd7d324ed2,0,Xiao Li was rescued when the local council rai...,d5c68b3bf,4.0,3
1,9789,9789,1,Puppy Girl,3,Mixed Breed,Rottweiler,2,Black,Brown,NaN,2,1,1,1,2,1,1,0,Kuala Lumpur,b770bac0ca797cf1433c48a35d30c4cb,0,Active and healthy. Already vaccine and deworm...,b968b85c1,2.0,3
2,11994,11994,2,Cookie And Cream,2,Domestic Short Hair,NaN,3,Yellow,Cream,White,1,1,2,1,2,1,2,0,Selangor,3b36a2f0ba551f7d0276cd1741369c37,0,These kittens were found behind the alley of m...,0cf645af3,3.0,1
3,5451,5451,1,Suki,36,Maltese,Poodle,2,White,NaN,NaN,1,2,1,1,1,1,1,100,Selangor,a63ec365c8c0d28ff01d2cb1b31406cb,0,Friendly..loving..love to play..potty trained,804e56f9b,4.0,1
4,9845,9845,1,A3,1,Mixed Breed,Mixed Breed,1,Black,NaN,NaN,2,2,2,1,2,1,1,0,Selangor,eba983b85e34963999d476ccada91619,0,For Adoption,b414c5dd9,3.0,1


In [22]:
df = df.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'RescuerID', 'PetID', 'Color2', 'Color3','Breed2','Name', 'VideoAmt', 'Description'])
df.head()

,Type,Age,Breed1,Gender,Color1,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,PhotoAmt,AdoptionSpeed
0,2,2,Domestic Short Hair,2,Brown,2,1,2,1,2,1,1,0,Selangor,4.0,3
1,1,3,Mixed Breed,2,Black,2,1,1,1,2,1,1,0,Kuala Lumpur,2.0,3
2,2,2,Domestic Short Hair,3,Yellow,1,1,2,1,2,1,2,0,Selangor,3.0,1
3,1,36,Maltese,2,White,1,2,1,1,1,1,1,100,Selangor,4.0,1
4,1,1,Mixed Breed,1,Black,2,2,2,1,2,1,1,0,Selangor,3.0,1


In [23]:
df.dtypes

Type               int64
Age                int64
Breed1            object
Gender             int64
Color1            object
MaturitySize       int64
FurLength          int64
Vaccinated         int64
Dewormed           int64
Sterilized         int64
Health             int64
Quantity           int64
Fee                int64
State             object
PhotoAmt         float64
AdoptionSpeed      int64
dtype: object

In [24]:
df['Type'] = df['Type'].astype('object')
df['Gender'] = df['Gender'].astype('object')
df['Vaccinated'] = df['Vaccinated'].astype('object')
df['Dewormed'] = df['Dewormed'].astype('object')
df['Sterilized'] = df['Sterilized'].astype('object')
df.dtypes

Type              object
Age                int64
Breed1            object
Gender            object
Color1            object
MaturitySize       int64
FurLength          int64
Vaccinated        object
Dewormed          object
Sterilized        object
Health             int64
Quantity           int64
Fee                int64
State             object
PhotoAmt         float64
AdoptionSpeed      int64
dtype: object

In [25]:
# df is the entire dataset with all predictors.
data = df.copy()
relief_data = df[["Sterilized","State", "Dewormed", "Breed1","AdoptionSpeed"]]

target_relief = relief_data.pop("AdoptionSpeed")
target = data.pop("AdoptionSpeed")

In [26]:
data.head()

,Type,Age,Breed1,Gender,Color1,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,PhotoAmt
0,2,2,Domestic Short Hair,2,Brown,2,1,2,1,2,1,1,0,Selangor,4.0
1,1,3,Mixed Breed,2,Black,2,1,1,1,2,1,1,0,Kuala Lumpur,2.0
2,2,2,Domestic Short Hair,3,Yellow,1,1,2,1,2,1,2,0,Selangor,3.0
3,1,36,Maltese,2,White,1,2,1,1,1,1,1,100,Selangor,4.0
4,1,1,Mixed Breed,1,Black,2,2,2,1,2,1,1,0,Selangor,3.0


In [27]:
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(data)
categorical_columns = categorical_columns_selector(data)

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

preprocessor = ColumnTransformer(
    [
        ("one-hot-encoder", categorical_preprocessor, categorical_columns),
        ("standard_scaler", numerical_preprocessor, numerical_columns),
    ]
)

model = make_pipeline(preprocessor, LogisticRegression(max_iter=500))
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Type', 'Breed1', 'Gender',
                                                   'Color1', 'Vaccinated',
                                                   'Dewormed', 'Sterilized',
                                                   'State']),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  ['Age', 'MaturitySize',
                                                   'FurLength', 'Health',
                                                   'Quantity', 'Fee',
                                                   'PhotoAmt'])])),
                ('logisticregression', LogisticRegression(max_iter=500))])

In [28]:
data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=42
)

In [29]:
_ = model.fit(data_train, target_train)


In [30]:
cv_results = cross_validate(model, data, target, cv=5)
scores = cv_results["test_score"]
print(
    "The mean cross-validation accuracy is: "
    f"{scores.mean():.3f} ± {scores.std():.3f}"
)

The mean cross-validation accuracy is: 0.365 ± 0.005


Applying the relief predictors

In [31]:
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(relief_data)
categorical_columns = categorical_columns_selector(relief_data)

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

preprocessor = ColumnTransformer(
    [
        ("one-hot-encoder", categorical_preprocessor, categorical_columns),
        ("standard_scaler", numerical_preprocessor, numerical_columns),
    ]
)

model = make_pipeline(preprocessor, LogisticRegression(max_iter=500))
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Sterilized', 'State',
                                                   'Dewormed', 'Breed1']),
                                                 ('standard_scaler',
                                                  StandardScaler(), [])])),
                ('logisticregression', LogisticRegression(max_iter=500))])

In [32]:
data_train, data_test, target_train, target_test = train_test_split(
    relief_data, target_relief, random_state=42
)

In [33]:
_ = model.fit(data_train, target_train)
cv_results = cross_validate(model, relief_data, target_relief, cv=5)
scores = cv_results["test_score"]
print(
    "The mean cross-validation accuracy is: "
    f"{scores.mean():.3f} ± {scores.std():.3f}"
)

The mean cross-validation accuracy is: 0.354 ± 0.004
